In [1]:
#--------------------------------
## IMPORTS
#--------------------------------
import sys
import os

# set the path to find the modules
sys.path.insert(0, '../src/') #use relative path
os.chdir("../005_src")

from config import *

----------------------------------------
Packages import successful
Variables import successful
Helper Functions import successful
Functions import successful
Functions xml import successful
Functions graph import successful
Functions GCN import successful
Functions Plotting import successful
Functions Data Adjustments import successful
----------------------------------------
root directory: /storage/remote/atcremers50/ss21_multiagentcontrol/
input directory: /storage/remote/atcremers50/ss21_multiagentcontrol/004_data/
output directory: /storage/remote/atcremers50/ss21_multiagentcontrol/006_model_output/
----------------------------------------

----------------------------------------
root directory: /storage/remote/atcremers50/ss21_multiagentcontrol/
input directory: /storage/remote/atcremers50/ss21_multiagentcontrol/004_data/
output directory: /storage/remote/atcremers50/ss21_multiagentcontrol/006_model_output/
----------------------------------------



In [2]:
#--------------------------------
## DEBUGGING OPTIONS
#--------------------------------
# mark True during dubugging to trigger the prints
printstat = True 

In [3]:
#--------------------------------
## INPUT 
#--------------------------------
# TODO: make it to select from xml or csv or pkl df
# select input xml 
xml_data = "../data/xml/sumoout_smallerCross.xml"

# sim variables 
edge_creation_radius = 30 # if the distance between two vehicles is less than this, and edge is created
edge_maintenance_radius = 70 # if the distance between two vehicles is less than this, and edge is maintened
simulation_max_duration = 100 # Int or None

# saving variables
savestat = False
delete_tempFiles = True
date = get_date()
ts = get_timestamp()
SAVE_TEMP = f"../data/figures/graph/{date}{ts}"
Path(SAVE_TEMP).mkdir(parents=True, exist_ok=True)
minl = 6

In [4]:
#--------------------------------
# MAIN PIPELINE
#--------------------------------

# read file into a dataframe
df_raw = xml_to_df(xml_data)
# custom formatting TODO: check if there is more useful stuff to do here
df = format_df(df_raw)

# check all vehicles in the sim
unique_vehicles = df.vehID.unique().tolist()
printif(f"there are {len(unique_vehicles)} unique vehicles" , printstat) 

# decide to mask eventually for shorter simulations
all_timesteps = list(df.time.unique())
if simulation_max_duration is None:
    limit_duration =  len(all_timesteps)
else:
    limit_duration = int(min(simulation_max_duration,len(all_timesteps)))
df_test = mask_timestep(df,all_timesteps[:limit_duration])
printif(f"the simulation duration is {limit_duration} timesteps" , printstat) 

# get one df with distances per each time step
printif(f"computing mutual distances between vehicle per time step" , printstat) 
df_list = get_df_per_timestep(df_test, 
                            edge_creation_radius = edge_creation_radius,
                            edge_maintenance_radius = edge_maintenance_radius
                            )
df_alltimestep_all_distances = pd.concat(df_list)
df_complete = df_alltimestep_all_distances.copy(deep=True)
df_with_edges = add_edge_info_per_df(df_complete)
printif(f"added edge information given ec={edge_creation_radius} and em= {edge_maintenance_radius}" , printstat) 

# save as pickle file
df_with_edges_path = f"../data/dataframes/{date}{ts}df_with_edges_{len(df_with_edges)}_ec{edge_creation_radius}_em{edge_maintenance_radius}.pkl"
df_with_edges.to_pickle(df_with_edges_path)
printif(f"saved df with edge information at {df_with_edges_path}" , printstat) 

there are 500 unique vehicles
the simulation duration is 100 timesteps
computing mutual distances between vehicle per time step
../src/configuration/_05_functions_graph.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["edge"] = edges_list
added edge information given ec=30 and em= 70
saved df with edge information at ../data/dataframes/20210620-16h23m49sdf_with_edges_3862_ec30_em70.pkl


In [5]:
#edit the timestep
#df = df_with_edges
df_with_edges["timestep"] = pd.to_numeric(df_with_edges["timestep"])
df_with_edges.sort_values("timestep")


# build the graph, store the pictures, return dictionary per timestep with edge list
dict_edges_per_frame,titleGif = build_graph(df_with_edges,
                                    ec = edge_creation_radius,
                                    em = edge_maintenance_radius,
                                    SAVE_TEMP= SAVE_TEMP,
                                    date = date,
                                    ts= ts,
                                    savestat = savestat, 
                                    delete_tempFiles= delete_tempFiles,
                                    minl= minl)

printif("building the graph and returing edges", printstat)
if savestat:
    print (f"graph visualization saved in {titleGif}")

print ()
print (dict_edges_per_frame[50])

building the graph and returing edges

[[('10', '15'), ('15', '5'), ('5', '7'), ('10', '7'), ('14', '2'), ('2', '5'), ('10', '2'), ('10', '16'), ('13', '15'), ('10', '13'), ('15', '7'), ('10', '5'), ('2', '7'), ('13', '16'), ('16', '7'), ('15', '16'), ('13', '7')]]


In [6]:
df_test = change_columns_format_to_numeric(df_test,"time")
df_test

../src/configuration/_03_functions_dataframes.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columns_numeric] = df[columns_numeric].apply(pd.to_numeric)


,time,vehID,X,Y,yaw,type,speed,pos,lane,slope
0,0.0,0,92.00,194.90,180.00,DEFAULT_VEHTYPE,0.00,5.10,4i_0,0.0
1,1.0,0,92.00,192.98,180.00,DEFAULT_VEHTYPE,1.92,7.02,4i_0,0.0
2,2.0,0,92.00,189.34,180.00,DEFAULT_VEHTYPE,3.64,10.66,4i_0,0.0
3,3.0,0,92.00,183.70,180.00,DEFAULT_VEHTYPE,5.64,16.30,4i_0,0.0
4,3.0,1,92.00,194.90,180.00,DEFAULT_VEHTYPE,0.00,5.10,4i_0,0.0
...,...,...,...,...,...,...,...,...,...,...
896,99.0,20,85.07,101.60,320.62,DEFAULT_VEHTYPE,3.29,1.33,1o_2,0.0
897,99.0,21,108.00,77.55,0.00,DEFAULT_VEHTYPE,0.87,77.55,3i_0,0.0
898,99.0,22,129.00,108.00,270.00,DEFAULT_VEHTYPE,13.13,71.00,2i_0,0.0
899,99.0,23,108.00,46.41,0.00,DEFAULT_VEHTYPE,12.14,46.41,3i_0,0.0


In [16]:
mask = (df_with_edges.edge == True)
df_with_edges[df_with_edges.timestep == 3]
df_with_edges = change_columns_format_to_numeric(df_with_edges,"veh_a")
df_with_edges = change_columns_format_to_numeric(df_with_edges,"veh_b")

In [17]:
for timestep in sorted(df_with_edges[mask].timestep.unique())[:1]:
    
    # filter the big datasets, taking only the timestep of interest
    df_features = df_test[df_test.time ==  timestep]
    df_edges = df_with_edges[df_with_edges.timestep == timestep]
    
    print (df_features)

    # get the vehicles with edges in this timestep
    unique_veh_a = np.unique(df_edges[['veh_a']].values).tolist()
    unique_veh_b = np.unique(df_edges[['veh_b']].values).tolist()
    unique_veh = list(set(unique_veh_a+unique_veh_b))
    printif(f"there are {len(unique_veh)} unique vehicles in timestep {timestep}" , printstat) 

    # for each unique vehicle extract the features and the positions
    data_x = []
    data_pos = []
    for veh in unique_veh:
        df_veh = mask_veh(df_features,veh)
        data_x.append(df_veh[['yaw','speed']].values.tolist()[0])
        data_pos.append(df_veh[['X','Y']].values.tolist()[0])
        
        #print (df_features["(xa,ya)"].values)
    
    # get all the edges from the edges df
    data_edges = [df_edges.veh_a.tolist(),df_edges.veh_b.tolist()]
    print (data_edges)

    # convert to tensor!
    data_x = torch.FloatTensor(data_x)
    data_pos = torch.FloatTensor(data_pos)
    data_edges = torch.FloatTensor(data_edges)

    




   time vehID     X      Y    yaw             type  speed   pos  lane  slope
3   3.0     0  92.0  183.7  180.0  DEFAULT_VEHTYPE   5.64  16.3  4i_0    0.0
4   3.0     1  92.0  194.9  180.0  DEFAULT_VEHTYPE   0.00   5.1  4i_0    0.0
there are 2 unique vehicles in timestep 3.0
[[0], [1]]
